# Joint Feldman DKG

as seen in [Secure Distributed Key Generation
for Discrete-Log Based Cryptosystems](https://link.springer.com/content/pdf/10.1007/3-540-48910-X_21.pdf)

## First part



In [1]:
import random as rnd #not secure, use os.random() instead

In [23]:
def create_polynomial(t,mod):
    coeff = [rnd.randint(1,mod-1) for _ in range(t+1)]
    return coeff

def create_commits(coeff,t,g,mod):
    comms = [pow(g,coeff[k],mod) for k in range(t+1)]
    return comms


In [3]:
def evaluate_polynomial(coeff,val,mod):
    s = 0
    for i in range(len(coeff)):
        s += (coeff[i]*pow(val,i)) %mod
        s=s%mod
    
    return s

In [4]:
def create_shares(coeff,n,mod,i):
    """
    Every party i creates shares for all j
    """
    evals = []
    for j in range(0,n):
        if j != i:
            val = evaluate_polynomial(coeff, j,mod)
            evals.append(val % mod)
        else:
            evals.append(0)
    return evals

In [19]:
def verify_share(share, comms, j, g, t, mod):
    """
    Given a share from party i and the commitment from that party,
    this function verifies that the share is valid.
    """
    flag = False

    # Compute G = g^share mod mod
    G = pow(g, share,mod)

    # Initialize the product s to 1
    s = 1

    # Compute the product of C^j^k for k in [0, t]
    for k in range(t+1):
        jk = pow(j, k)
        Cijk = pow(comms[k], jk,mod)
        s *= Cijk
        s = s % mod

    # Check if G == s, and set the flag accordingly
    if G == s:
        print("yeah!")
        flag = True
    else:
        print(f"G={G}, s={s}")

    return flag



def verify_shares(shares,comms,j,i,g,mod,n,t):
    j_share=0
    j_QUAL = [j] # each party will assume it is right
    
    
    for w in range(n):
        if w != j:
            # get share from party w
            j_share = shares["P_"+str(w)][j]
    
            # compute product of commitments
            s = 1
            comms_vec=comms["P_"+str(w)]
            print()
            print("P_"+str(w)+": ",comms_vec)
            # comms_vec.reverse()
            for k in range(0,t+1):
                s *= comms_vec[k]**((w)**k)
                # s *= comms["P_"+str(i)][k]**((i)**k)
            
            # assert equality
            if pow(g,j_share,mod) == (s % mod):
                j_QUAL.append(w)
            else:
                print(w,j_share)
                print("sum", s%mod,sep='\t')
                print("g**si", pow(g,j_share,mod),sep='\t')
    
    j_QUAL.sort()
    return (j,j_QUAL)

In [21]:
t=5; n=10
a=0; b=100
g = 3; 
p=23
q=11 # q|p-1
#r = 6361

In [22]:
coeff = {}
for i in range(n):
    coeff["P_"+str(i)] = create_polynomial(t,q)

print("coeff")
coeff

coeff


{'P_0': [5, 4, 10, 7, 10, 9],
 'P_1': [7, 4, 4, 1, 1, 2],
 'P_2': [3, 2, 6, 1, 7, 7],
 'P_3': [1, 6, 10, 10, 7, 10],
 'P_4': [4, 8, 6, 7, 8, 4],
 'P_5': [4, 10, 5, 4, 9, 9],
 'P_6': [10, 6, 9, 2, 3, 6],
 'P_7': [7, 5, 7, 5, 8, 7],
 'P_8': [7, 8, 1, 6, 10, 10],
 'P_9': [6, 5, 9, 1, 6, 9]}

In [24]:
comms = {}
for i in range(n):
    comms["P_"+str(i)] = create_commits(coeff["P_"+str(i)],t,g,p)
comms

{'P_0': [13, 12, 8, 2, 8, 18],
 'P_1': [2, 12, 12, 3, 3, 9],
 'P_2': [4, 9, 16, 3, 2, 2],
 'P_3': [3, 16, 8, 8, 2, 8],
 'P_4': [12, 6, 16, 2, 6, 12],
 'P_5': [12, 8, 13, 12, 18, 18],
 'P_6': [8, 16, 18, 9, 4, 16],
 'P_7': [2, 13, 2, 13, 6, 2],
 'P_8': [2, 6, 3, 16, 8, 8],
 'P_9': [16, 13, 18, 3, 16, 18]}

In [25]:
shares = {}
for i in range(n):
    shares["P_"+str(i)] = create_shares(coeff["P_"+str(i)],n,q,i)

print("shares:")
shares

shares:


{'P_0': [0, 1, 7, 4, 8, 6, 2, 8, 2, 7],
 'P_1': [7, 0, 9, 0, 2, 10, 2, 3, 6, 3],
 'P_2': [3, 4, 0, 4, 1, 6, 8, 8, 1, 2],
 'P_3': [1, 0, 4, 0, 9, 9, 3, 3, 8, 5],
 'P_4': [4, 4, 4, 10, 0, 1, 0, 3, 5, 0],
 'P_5': [4, 8, 2, 1, 9, 0, 3, 4, 4, 4],
 'P_6': [10, 3, 6, 5, 2, 9, 0, 5, 3, 6],
 'P_7': [7, 6, 8, 6, 6, 9, 4, 0, 0, 10],
 'P_8': [7, 9, 5, 10, 6, 9, 4, 1, 0, 7],
 'P_9': [6, 3, 4, 8, 8, 4, 5, 9, 5, 0]}

In [26]:
j=3

for i in range(n):
    if i != j:
        share = shares['P_'+str(i)][j]
        print(f"share {i}: {share}")
        comms_i = comms["P_"+str(i)]
        print("comms", comms_i)
        print(verify_share(share,comms_i,j, g,t, p))
        print()
        # j,j_QUAL = verify_shares(shares,comms,j,i,g,r,n,t)
        # j_QUAL

share 0: 4
comms [13, 12, 8, 2, 8, 18]
yeah!
True

share 1: 0
comms [2, 12, 12, 3, 3, 9]
yeah!
True

share 2: 4
comms [4, 9, 16, 3, 2, 2]
yeah!
True

share 4: 10
comms [12, 6, 16, 2, 6, 12]
yeah!
True

share 5: 1
comms [12, 8, 13, 12, 18, 18]
yeah!
True

share 6: 5
comms [8, 16, 18, 9, 4, 16]
yeah!
True

share 7: 6
comms [2, 13, 2, 13, 6, 2]
yeah!
True

share 8: 10
comms [2, 6, 3, 16, 8, 8]
yeah!
True

share 9: 8
comms [16, 13, 18, 3, 16, 18]
yeah!
True



In [11]:
i = 2
evals=shares["P_"+str(i)]
print("eval",evals,sep='\t')

s = 1

for k in range(0,t+1):
    s *= comms["P_"+str(i)][k]**((j)**k)

    
print("sum", s%r,sep='\t')
print("g**si", pow(g,int(evals[i]),r),sep='\t')

assert s%r == g**evals[i]%r




eval	[1, 4, 0, 0, 3, 6, 9, 9, 11, 0]
sum	3
g**si	1


AssertionError: 

## Second part

## Third part

In [ ]:
a = [3,6,1]
a.sort()
a

In [ ]:
pow(2,10,13)


In [ ]:
3**7 %13
81%13
